# The Forward Pass

Having completed work on the symmetry function calculation, despite the disappointing overall timing, we can move on to the forward pass, written in a way that either the individual or total energies can be calculated given the input parameters and the symmetry functions. This either means passing individual vectors, or a full matrix, though passing a matrix is a question for Alex.

In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,DelimitedFiles,LinearAlgebra
@everywhere using InvertedIndices
@everywhere using MachineLearningPotential

We need to define a _struct_ containing the actual neural network parameters. In general this should contain the integer num_layers as well as vectors of length num_layers num_nodes and activation_functions. We use the num_nodes vector to define the num_parameters and this gives us the vector length for parameters, the actual values contained in the neural network. 

In [2]:
struct NeuralNetworkPotential
    n_layers::Int32
    n_params::Int32
    num_nodes::Vector
    activation_functions::Vector
    parameters::Vector
end

function NeuralNetworkPotential(num_nodes::Vector,activation_functions::Vector, parameters)
    
    return NeuralNetworkPotential(length(num_nodes),length(parameters),num_nodes,activation_functions,parameters)
end

# function forward_pass(eatom,input,batchsize,)

NeuralNetworkPotential

In [24]:
function forward_pass( input::AbstractArray, batchsize, num_layers, num_nodes, activation_functions, num_parameters, parameters)    
    eatom = Vector{Float64}(undef,batchsize)
    ccall( (:forward, "./librunnerjulia.so"),
    Float64,  (Ref{Float64},Ref{Int32}, Ref{Int32}, Ref{Int32}, Ref{Int32},
    Ref{Int32}, Ref{Float64}, Ref{Int32}, Ref{Float64}),
    input, num_nodes[1], num_layers, num_nodes, batchsize,
    num_parameters, parameters, activation_functions, eatom
    )
    return eatom
end

function forward_pass(input::AbstractArray,batchsize,nnparams::NeuralNetworkPotential)   
    return forward_pass(input, batchsize, nnparams.n_layers, nnparams.num_nodes, nnparams.activation_functions,nnparams.n_params, nnparams.parameters)
end

function forward_pass(eatom,input::AbstractArray,batchsize,nnparams::NeuralNetworkPotential)   
    return forward_pass(eatom,input, batchsize, nnparams.n_layers, nnparams.num_nodes, nnparams.activation_functions,nnparams.n_params, nnparams.parameters)
end

function forward_pass( eatom,input::AbstractArray, batchsize, num_layers, num_nodes, activation_functions, num_parameters, parameters)    
    ccall( (:forward, "./librunnerjulia.so"),
    Float64,  (Ref{Float64},Ref{Int32}, Ref{Int32}, Ref{Int32}, Ref{Int32},
    Ref{Int32}, Ref{Float64}, Ref{Int32}, Ref{Float64}),
    input, num_nodes[1], num_layers, num_nodes, batchsize,
    num_parameters, parameters, activation_functions, eatom
    )
    return eatom
end

forward_pass (generic function with 4 methods)

In [25]:
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open("weights.029.data","r+")
weights=readdlm(file)
close(file)
weights = vec(weights)
gauge_thing = NeuralNetworkPotential(num_nodes,activation_functions,weights)

NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841])

In [26]:
file = open("$(pwd())/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])
truevals = Matrix{Float64}(truevals)

file = open("$(pwd())/scaling.data")
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
renormalised_truevals = copy(truevals)
for g_index in eachindex(G_value_vec)
    renormalised_truevals[g_index,:] .-=  G_value_vec[g_index][2]
    renormalised_truevals[g_index,:] = renormalised_truevals[g_index,:] ./ (G_value_vec[g_index][1] - G_value_vec[g_index][2])
end

# Test the Forward Pass
writing the forward pass function, time to see whether it returns the correct energies

In [27]:
E = forward_pass(renormalised_truevals,55,gauge_thing)
sum(E)

-6.216814248768345

In [28]:
E = forward_pass(E,renormalised_truevals,55,gauge_thing)
sum(E)

-6.216814248768345

In [31]:
"$(pwd())/"

"/home/ghunter/.julia/dev/MachineLearningPotential/scripts/"